# Flow development lifecycle

**Goal** - This is a sample notebook to show how to development a prompt-flow then use it in a pipeline like a component. 
Please note the function is not fully ready. Currently it's just to show the E2E user experience.

## 0. Learn prompt-flow knowledge and prepare environment through this

Please follow [doc site](https://promptflow.azurewebsites.net/index.html) to learn the concepts and samples of prompt flow.

## Flow Development Opetion 1: Debug locally
You can author the promptflow and debug/run it locally.

Flow local test and batch run in SDK:

In [ ]:
import json
from promptflow import PFClient

# client can help manage your runs and connections.
pf = PFClient()

flow = "../../flows/standard/standard-flow"  # path to the flow directory

flow_inputs =   {
    "text": "Hello World!"
  }

flow_result = pf.test(flow=flow, inputs=flow_inputs)

print(f"Flow result: {flow_result}")

batch_run_inputs = "../../flows/standard/standard-flow/data.jsonl"

batch_run = pf.run(
    flow=flow, 
    data = batch_run_inputs, 
    inputs=batch_run_inputs, 
    column_mapping={
        "text": "${data.text}",
    },  
    stream=True,
)

#pf.visualize(batch_run)



Flow local test and batch run in CLI:

In [ ]:
# test with one line input
!pf flow test --flow ../../flows/standard/standard-flow --inputs text="Hello CLI!"

# batch run with jsonl file locally
!pf run create --flow ../../flows/standard/standard-flow --data ../../flows/standard/standard-flow/data.jsonl  --stream

## Flow Development Option 2: Debug in eyes-on environment
Once the local debug is done, or you need to test the flow with data/connection in cloud, You can submit the run to eyes-on environment.

In [ ]:
# Import required libraries
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential

from azure.ai.ml import MLClient, Input
from azure.ai.ml.constants import AssetTypes
from azure.ai.ml.dsl import pipeline
from azure.ai.ml import load_component
from promptflow.azure import PFClient

try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    credential = InteractiveBrowserCredential()

# Get a handle to workspace
ml_client = MLClient.from_config(credential=credential)

# Create PFClient connected to workspace
pf = PFClient(ml_client)

flow = "../../flows/standard/standard-flow" 
batch_run_inputs = "../../flows/standard/standard-flow/data.jsonl"
runtime= "[SomeRuntimeName]"
batch_run = pf.run(flow=flow,data=batch_run_inputs,runtime=runtime)
print(batch_run)

run in CLI:

In [ ]:
!pfazure run create --flow ../../flows/standard/standard-flow --data ../../flows/standard/standard-flow/data.jsonl --runtime SomeRuntimeName

## Use the flow in a pipeline

Then you can use the tested flow along with other components in a pipeline seamless:

In [ ]:
from azure.ai.ml import load_component 
from azure.ai.ml import dsl 
 
flow_component = load_component("../../flows/standard/standard-flow/flow.dag.yaml") 
 
@dsl.pipeline 
def pipeline_with_flow(job_input): 
    # you can use it as an anonymous component in dsl component directly 
    flow_node = flow_component(data=job_input) 
    flow_node.compute = "cpu-cluster" 
    flow_node.logging_level = "DEBUG" 
    return {"job_output": flow_node.outputs.flow_outputs} 
 
pipeline_job = pipeline_with_flow(job_input=Input(path="../../flows/standard/standard-flow/data.jsonl")) 
ml_client.jobs.create_or_update(pipeline_job) 

created_job = ml_client.jobs.create_or_update(pipeline)

Register component to workspace

In [ ]:
# you can also register this component with specific name & version 
# if name is not found in source files, it must be set before registration and will be validated 
flow_component.name = "basic-standard-flow" 
# version must be provided in registration 
ml_client.components.create_or_update(flow_component, version="2") 
